# Libraries

In [1]:
import pandas as pd
import re

from thefuzz import process

/home/amanda/projects/2023-ds-jobdesc/venv-2023-ds-jobdesc/lib/python3.11/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Configurations

In [2]:
# Pandas configuration for long phrases
pd.set_option('display.max_colwidth', 50)

# Data

In [3]:
df = pd.read_csv('data/Jobs.csv')

In [4]:
df.head()

,Unnamed: 0,title,company,announcement,description
0,0,"Senior Analyst, Data Science and Analytics",TransUnion,The Muse,TransUnion's Job Applicant Privacy Notice Wha...
1,1,Senior Data Scientist,"Grubhub Holdings, Inc.",ZipRecruiter,About The Opportunity We're all about connect...
2,2,Lead Data Science Analyst,Discover Financial Services,LinkedIn,"Discover. A brighter future. With us, you’ll ..."
3,3,Data Science Intern,AbelsonTaylor,Startup Jobs,Are you a 2023 college graduate or rising coll...
4,4,Data Scientist,NORC at the University of Chicago,SimplyHired,"JOB DESCRIPTION: At NORC, Data Scientists pla..."


# Cleaning Data

In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [6]:
df.head()

,title,company,announcement,description
0,"Senior Analyst, Data Science and Analytics",TransUnion,The Muse,TransUnion's Job Applicant Privacy Notice Wha...
1,Senior Data Scientist,"Grubhub Holdings, Inc.",ZipRecruiter,About The Opportunity We're all about connect...
2,Lead Data Science Analyst,Discover Financial Services,LinkedIn,"Discover. A brighter future. With us, you’ll ..."
3,Data Science Intern,AbelsonTaylor,Startup Jobs,Are you a 2023 college graduate or rising coll...
4,Data Scientist,NORC at the University of Chicago,SimplyHired,"JOB DESCRIPTION: At NORC, Data Scientists pla..."


In [7]:
# Data types of columns
df.dtypes

title           object
company         object
announcement    object
description     object
dtype: object

In [8]:
# Dataframe information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         790 non-null    object
 1   company       790 non-null    object
 2   announcement  790 non-null    object
 3   description   790 non-null    object
dtypes: object(4)
memory usage: 24.8+ KB


In [9]:
# Transforming all string in lower case
df = df.apply(lambda x: x.str.lower())

In [10]:
# Get duplicated values
df[df.duplicated()]

,title,company,announcement,description


In [11]:
# Missing values
df[df['description'].isna()]

# There is no NA values. 

,title,company,announcement,description


In [12]:
# All type of unique titles of the job
df['title'].nunique()

513

In [13]:
df['title'].unique()

array(['senior analyst, data science and analytics',
       'senior data scientist', 'lead data science analyst',
       'data science intern', 'data scientist',
       'senior solutions architect (data science)',
       'data scientist - research, development & construction',
       'data science manager - s&a strategy',
       'senior/principal data scientist', 'data science analyst',
       'senior data scientist - knowledge management',
       'data scientist, consultant', 'senior-data scientist',
       'staff data scientist', 'data science internship',
       'undergrad intern – data science program (summer 2023)',
       'data science, department of information technology - adjunct faculty',
       'summer 2023 intelligent sensors data science intern',
       'associate director data science',
       'data science senior advisor (solution value analytics) - evernorth',
       'director, data science',
       'senior manager, institutional data analytics and reporting',
       'd

There are 515 unique job title, but the most are repeated, so we have to classify this titles in some unique occurences. 

# Creating levels based on job title

## Filters for data analyst and data scientist

In [14]:
# Checking if the tile of the job contains analyst

job_analyst = df[df['title'].str.contains('analyst')]
job_scientist = df[df['title'].str.contains('scientist|science')]

In [15]:
# Checking what is duplicated between job_analyst and job_scientist

title_analyst_scientist = set(job_analyst['title']).intersection(job_scientist['title'])

job_analyst_or_scientist = pd.DataFrame()

for i in title_analyst_scientist:
    job_analyst_or_scientist = pd.concat([job_analyst_or_scientist, df[df['title'] == i]])

In [16]:
# Now, I want to remove from job_anayst and job_scientist the in tersection cases

job_analyst = job_analyst[~job_analyst['title'].isin(job_analyst_or_scientist['title'])]
job_scientist = job_scientist[~job_scientist['title'].isin(job_analyst_or_scientist['title'])]

In [17]:
job_scientist['title'].str.contains('principal')

1      False
3      False
4      False
5      False
6      False
       ...  
326    False
584    False
610     True
694    False
700    False
Name: title, Length: 315, dtype: bool

## Creating levels 

In [18]:
# Function that creates the levels of jobs, respecting this relationship
# level 1 - intern position
# level 2 - junior position
# level 3 - intermediate position
# level 4 - senior position
# level 5 - principal position
# level 6 - leader position

def job_level(df):
    df['level'] = df.apply(lambda x:
                           1 if 'intern' in x['title'] or 'internship' in x['title'] else
                           2 if 'junior' in x['title'] or 'associate' in x['title'] or re.search(r'\bi\b', x['title']) else
                           4 if 'senior' in x['title'] or 'sr' in x['title'] or 'snr' in x['title'] or re.search(r'\biii\b', x['title']) else
                           5 if 'principal' in x['title'] else
                           6 if 'lead' in x['title'] else
                           3, axis=1)
    
    return df

Now we saw that there is 6 unique job titles. 

In [19]:
# Applying the levels on the dataframes 
job_analyst = job_level(job_analyst).reset_index(drop= True)
job_scientist = job_level(job_scientist).reset_index(drop= True)
job_analyst_or_scientist = job_level(job_analyst_or_scientist).reset_index(drop= True)

In [20]:
job_analyst.head()

,title,company,announcement,description,level
0,predictive data analyst,county of los angeles,government jobs,type of recruitment: open competitive job oppo...,3
1,data analyst,robert half,linkedin,job title: data analyst location: saddle broo...,3
2,senior business data analyst,matrix resources,matrix resources,work with the brightest minds at one of the la...,4
3,junior data analyst,ifg - international financial group,linkedin,"title: junior data analyst location: redmond,...",2
4,data analyst (monitoring),general dynamics information technology,gdit,the data analyst (monitoring) will support the...,3


In [21]:
job_scientist.head()

,title,company,announcement,description,level
0,senior data scientist,"grubhub holdings, inc.",ziprecruiter,about the opportunity we're all about connect...,4
1,data science intern,abelsontaylor,startup jobs,are you a 2023 college graduate or rising coll...,1
2,data scientist,norc at the university of chicago,simplyhired,"job description: at norc, data scientists pla...",3
3,senior solutions architect (data science),the trade desk,linkedin,the trade desk is a global technology company ...,4
4,senior data scientist,tango card,startup jobs,the jobtango card is hiring our first ever sen...,4


In [22]:
job_analyst_or_scientist

,title,company,announcement,description,level
0,data scientist (data analyst/business intellig...,marathon petroleum corporation (mpc),ziprecruiter,"an exciting career awaits you at mpc, we're c...",3
1,"senior analyst, data science and analytics",transunion,the muse,transunion's job applicant privacy notice wha...,4
2,"senior analyst, data science and analytics",hr&a advisors,hr&a advisors - jazzhr,"about us | hr&a advisors, inc. (hr&a) is an ...",4
3,data analyst/data scientist,epam systems,epam,"we are hiring a data business analyst, who wil...",3
4,lead data science analyst,discover financial services,linkedin,"discover. a brighter future. with us, you’ll ...",6
5,data scientist / analyst,nft text inc.,linkedin,"we're looking for an entrepreneurial, data-obs...",3
6,data science analyst,jlt employee benefits,adzuna,"at mercer, we deliver innovative solutions tha...",3
7,data science analyst,bunge north america,careers in food,"at bunge, people don t just come here to work,...",3
8,data science analyst,american heart association,american heart association jobs,overview now is the time to join us and make ...,3
9,data science analyst,american heart association,american heart association jobs,overview now is the time to join us and make ...,3


# Opening the description field

And what I want to find in the description field? 
- Average salary in year
- Three most requested tools and libraries 
- Most requested education level
- Most common tasks

## Most requested tools and libraries

In [40]:
tools_libraries = {'python': ['python', 'jupyter', 'numpy', 'pandas', 'matplotlib', 'seaborn', 'scipy',
                            'scikit-learn', 'tensorflow', 'pytorch', 'keras', 'xgboost', 'lightgbm', 'plotly',
                            'dask', 'nltk', 'opencv', 'gensim', 'statsmodel', 'theano', 'caffe', 'keras-tuner',
                            'auto-keras', 'autosklearn', 'shap', 'eli5', 'bokeh', 'folium', 'ggplot', 'plotnine', 
                            'geopandas', 'datashader', 'yellowbrick', 'flask', 'dash', 'streamlit', 
                            'fastapi', 'pyspark','tensorboard', 'cudf', 'networkx', 'beautifulsoup', 'scrapy', 
                            'numba', 'cython'],
                    'r': ['r'],
                    'sql': ['sql'],
                    'spark': ['spark'],
                    'hadoop': ['hadoop'],
                    'cloud': ['aws', 'google cloud', 'azure', 'ibm watson', 'h2o.ai']}

def find_tools(description):
    found_tools = []
    tool_library = []
    for tool, libraries in tools_libraries.items():
        libraries_found = [library for library in libraries if re.search(r'\b{}\b'.format(library), description)]
        if libraries_found:
            found_tools.append(tool)
            tool_library.append(libraries_found)
    return found_tools, sum(tool_library, [])

job_scientist[['tools', 'libraries']] = job_scientist['description'].apply(find_tools).apply(pd.Series)


In [41]:
job_scientist

,title,company,announcement,description,level,libraries,tools
0,senior data scientist,"grubhub holdings, inc.",ziprecruiter,about the opportunity we're all about connect...,4,"[python, numpy, pandas, scikit-learn, statsmod...","[python, sql, cloud]"
1,data science intern,abelsontaylor,startup jobs,are you a 2023 college graduate or rising coll...,1,[],[]
2,data scientist,norc at the university of chicago,simplyhired,"job description: at norc, data scientists pla...",3,"[plotly, r]","[python, r]"
3,senior solutions architect (data science),the trade desk,linkedin,the trade desk is a global technology company ...,4,"[python, r]","[python, r]"
4,senior data scientist,tango card,startup jobs,the jobtango card is hiring our first ever sen...,4,"[python, r, sql, aws]","[python, r, sql, cloud]"
...,...,...,...,...,...,...,...
310,lead data scientist - 100% remote - fte - $200...,dice,linkedin,dice is the leading career destination for tec...,6,[python],[python]
311,data scientist jobs,mantech international,clearance jobs,"secure our nation, ignite your future join th...",3,"[python, sql, hadoop]","[python, sql, hadoop]"
312,senior/principal scientist- visualization and ...,genentech,my stateline jobs,the position genentech (research and early de...,4,"[python, r]","[python, r]"
313,data science intern,geospark analytics inc.,simplyhired,company overview: join the world’s only threa...,1,[python],[python]


In [70]:
count = {tool: 0 for tool in tools_libraries.keys()}

for i in job_scientist['tools']:
    for tool in i:
        count[tool] += 1

count

{'python': 265, 'r': 179, 'sql': 177, 'spark': 62, 'hadoop': 44, 'cloud': 78}

In [98]:
count = {library: 0 for libraries in tools_libraries.values() for library in libraries}

for i in job_scientist['libraries']:
    for library in i:
        count[library] += 1

count

{'python': 258,
 'jupyter': 18,
 'numpy': 22,
 'pandas': 34,
 'matplotlib': 9,
 'seaborn': 3,
 'scipy': 5,
 'scikit-learn': 29,
 'tensorflow': 53,
 'pytorch': 43,
 'keras': 25,
 'xgboost': 6,
 'lightgbm': 1,
 'plotly': 5,
 'dask': 0,
 'nltk': 1,
 'opencv': 2,
 'gensim': 1,
 'statsmodel': 1,
 'theano': 0,
 'caffe': 3,
 'keras-tuner': 0,
 'auto-keras': 0,
 'autosklearn': 0,
 'shap': 0,
 'eli5': 0,
 'bokeh': 0,
 'folium': 0,
 'ggplot': 3,
 'plotnine': 0,
 'geopandas': 1,
 'datashader': 0,
 'yellowbrick': 0,
 'flask': 3,
 'dash': 5,
 'streamlit': 0,
 'fastapi': 0,
 'pyspark': 25,
 'tensorboard': 0,
 'cudf': 0,
 'networkx': 1,
 'beautifulsoup': 0,
 'scrapy': 0,
 'numba': 0,
 'cython': 0,
 'r': 179,
 'sql': 177,
 'spark': 62,
 'hadoop': 44,
 'aws': 61,
 'google cloud': 10,
 'azure': 34,
 'ibm watson': 1,
 'h2o.ai': 0}

In [112]:
# But inside each tool where there is more than one library, what's the most requested libraries? 

# Python

count = {libraries: 0 for libraries in tools_libraries['python']}

for i in job_scientist['libraries']:
    for library in i:
        if library in count.keys():
            count[library] += 1

count

{'python': 258,
 'jupyter': 18,
 'numpy': 22,
 'pandas': 34,
 'matplotlib': 9,
 'seaborn': 3,
 'scipy': 5,
 'scikit-learn': 29,
 'tensorflow': 53,
 'pytorch': 43,
 'keras': 25,
 'xgboost': 6,
 'lightgbm': 1,
 'plotly': 5,
 'dask': 0,
 'nltk': 1,
 'opencv': 2,
 'gensim': 1,
 'statsmodel': 1,
 'theano': 0,
 'caffe': 3,
 'keras-tuner': 0,
 'auto-keras': 0,
 'autosklearn': 0,
 'shap': 0,
 'eli5': 0,
 'bokeh': 0,
 'folium': 0,
 'ggplot': 3,
 'plotnine': 0,
 'geopandas': 1,
 'datashader': 0,
 'yellowbrick': 0,
 'flask': 3,
 'dash': 5,
 'streamlit': 0,
 'fastapi': 0,
 'pyspark': 25,
 'tensorboard': 0,
 'cudf': 0,
 'networkx': 1,
 'beautifulsoup': 0,
 'scrapy': 0,
 'numba': 0,
 'cython': 0}

In [119]:
# Cloud


count = {libraries: 0 for libraries in tools_libraries['cloud']}

for i in job_scientist['libraries']:
    for library in i:
        if library in count.keys():
            count[library] += 1

count

{'aws': 61, 'google cloud': 10, 'azure': 34, 'ibm watson': 1, 'h2o.ai': 0}

## Most requested tasks

In [145]:
count_tasks = {'python programming': 0, 'statistics': 0 ,'probability': 0, 'machine learning': 0,'data visualization': 0, 
                       'preprocessing':0, 'cleaning': 0, 'database management': 0, 'analysis': 0, 'modeling': 0, 'deep learning': 0,
                       'data engineering': 0, 'visualization': 0, 'manipulation': 0, 'machine learning': 0, 'storage': 0, 'cloud computing': 0, 
                       'etl': 0, 'warehousing': 0,'governance': 0, 'security': 0,'storytelling': 0, 'product development': 0, 
                       'natural language processing': 0, 'nlp': 0, 'computer vision': 0, 'business intelligence': 0, 'mining': 0,
                       'feature engineering': 0, 'time series analysis': 0, 'regression analysis': 0, 'classification algorithms': 0,
                       'clustering algorithms': 0, 'neural networks': 0, 'decision trees': 0, 'random forests': 0, 'support vector machines': 0, 
                       'svm': 0, 'k-nearest neighbors': 0, 'knn': 0, 'reinforcement learning': 0,'hyperparameter tuning': 0,
                       'ensemble learning': 0, 'transfer learning': 0, 'unsupervised learning': 0,'supervised learning': 0,
                       'exploratory data analysis': 0, 'eda': 0, 'quality control': 0, 'data interpretation': 0, 'collaboration': 0, 
                       'communication': 0, 'project management': 0, 'agile development': 0, 'software engineering': 0, 'version control': 0, 
                       'debugging': 0, 'troubleshooting': 0, 'ci/cd': 0, 'optimization': 0, 'deployment': 0}

for i in job_scientist['description']:
    for task in count_tasks.keys():
        if task in i:
            count_tasks[task] += 1

count_tasks_sorted = {k:v for k, v in sorted(count_tasks.items(), key = lambda x: x[1], reverse= True)}

count_tasks_sorted

{'analysis': 238,
 'machine learning': 206,
 'statistics': 197,
 'modeling': 160,
 'communication': 158,
 'visualization': 129,
 'deep learning': 77,
 'optimization': 74,
 'data visualization': 73,
 'mining': 72,
 'security': 60,
 'collaboration': 59,
 'deployment': 54,
 'natural language processing': 47,
 'nlp': 33,
 'data engineering': 31,
 'neural networks': 30,
 'computer vision': 28,
 'etl': 25,
 'project management': 24,
 'software engineering': 22,
 'cleaning': 21,
 'governance': 21,
 'business intelligence': 21,
 'exploratory data analysis': 21,
 'product development': 19,
 'probability': 17,
 'feature engineering': 17,
 'version control': 17,
 'decision trees': 15,
 'cloud computing': 14,
 'manipulation': 13,
 'storage': 13,
 'storytelling': 10,
 'supervised learning': 10,
 'eda': 10,
 'ci/cd': 10,
 'reinforcement learning': 9,
 'agile development': 9,
 'time series analysis': 8,
 'svm': 7,
 'unsupervised learning': 7,
 'quality control': 7,
 'debugging': 7,
 'regression analy

In [153]:
count_tasks_sorted['data engineering']

31

In [158]:
count = 0 
for i in job_scientist['description']:
    if 'data engineering' in i:
        count += 1
       

In [159]:
count

31